**Inputs**
- `experiment/resources/lists/con_run1.csv`
- `experiment/resources/lists/abs_run1.csv`
- `experiment/resources/lists/con_run2.csv`
- `experiment/resources/lists/abs_run2.csv`
- `stimuli_selection/lexicons/subtlex-pl.csv`
- `stimuli_selection/lexicons/anpw_r.csv`

**Outputs**
- `stimuli_selection/reports/stimuli_report_run1.csv`
- `stimuli_selection/reports/stimuli_report_run2.csv`
- `stimuli_selection/reports/stimuli_report_all.csv`
- `stimuli_selection/reports/stimuli_distributions/`


In [1]:
from __future__ import annotations

from pathlib import Path
import re
import math

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

def find_project_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for candidate in [p, *p.parents]:
        if (candidate / "experiment" / "resources" / "lists").exists():
            return candidate
    raise RuntimeError(
        "Project root not found. Expected folder: experiment/resources/lists "
        "somewhere above the current working directory."
    )

ROOT = find_project_root()

LISTS_DIR = ROOT / "experiment" / "resources" / "lists"
LEX_DIR = ROOT / "stimuli_selection" / "lexicons"
REPORTS_DIR = ROOT / "stimuli_selection" / "reports"
FIG_DIR = REPORTS_DIR / "figures"

REPORTS_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)


LIST_FILES = {
    "con_run1": LISTS_DIR / "con_run1.csv",
    "abs_run1": LISTS_DIR / "abs_run1.csv",
    "con_run2": LISTS_DIR / "con_run2.csv",
    "abs_run2": LISTS_DIR / "abs_run2.csv",
}

SUBTLEX_FILE = LEX_DIR / "subtlex-pl.csv"
ANPW_FILE = LEX_DIR / "anpw_r.csv"

CSV_ENCODING_LISTS = "utf-8-sig"


In [2]:
def _norm_word(x: str) -> str:
    return str(x).strip().lower()

def count_syllables_pl(word: str) -> int:
    """Very simple syllable count for Polish (heuristic)."""
    w = _norm_word(word)
    vowels = "aeiouyąęó"
    groups = re.findall(rf"[{vowels}]+", w)
    return max(1, len(groups)) if w else 0

def cohen_d(x: np.ndarray, y: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]
    if len(x) < 2 or len(y) < 2:
        return np.nan
    nx, ny = len(x), len(y)
    vx, vy = np.var(x, ddof=1), np.var(y, ddof=1)
    sp = math.sqrt(((nx - 1) * vx + (ny - 1) * vy) / (nx + ny - 2))
    return (np.mean(x) - np.mean(y)) / sp if sp > 0 else np.nan

def bootstrap_ci_d(x: np.ndarray, y: np.ndarray, n_boot: int = 3000, seed: int = 123) -> tuple[float, float]:
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]
    if len(x) < 2 or len(y) < 2:
        return (np.nan, np.nan)
    ds = []
    for _ in range(n_boot):
        xb = rng.choice(x, size=len(x), replace=True)
        yb = rng.choice(y, size=len(y), replace=True)
        ds.append(cohen_d(xb, yb))
    lo, hi = np.nanpercentile(ds, [2.5, 97.5])
    return float(lo), float(hi)


In [ ]:
# load trial lists

dfs = {}
for key, path in LIST_FILES.items():
    if not path.exists():
        raise FileNotFoundError(f"Missing list file: {path}")
    df = pd.read_csv(path, encoding=CSV_ENCODING_LISTS)
    if "word" not in df.columns:
        raise ValueError(f"{path} must contain 'word' column")
    df["list_name"] = key
    df["run"] = 1 if "run1" in key else 2
    df["condition"] = "CON" if key.startswith("con") else "ABS"
    df["word_norm"] = df["word"].map(_norm_word)
    dfs[key] = df

lists = pd.concat(dfs.values(), ignore_index=True)
lists.head(), lists.shape


(       word condition  run                          stimFile list_name  \
 0     zegar       CON    1  resources/audio/con_run1_001.wav  con_run1   
 1  samochód       CON    1  resources/audio/con_run1_002.wav  con_run1   
 2  grzebień       CON    1  resources/audio/con_run1_003.wav  con_run1   
 3     krowa       CON    1  resources/audio/con_run1_004.wav  con_run1   
 4    piasek       CON    1  resources/audio/con_run1_005.wav  con_run1   
 
   word_norm  
 0     zegar  
 1  samochód  
 2  grzebień  
 3     krowa  
 4    piasek  ,
 (96, 6))

In [ ]:
# load lexicons

if not SUBTLEX_FILE.exists():
    raise FileNotFoundError(f"Missing SUBTLEX file: {SUBTLEX_FILE}")
if not ANPW_FILE.exists():
    raise FileNotFoundError(f"Missing ANPW file: {ANPW_FILE}")

sub = pd.read_csv(SUBTLEX_FILE, sep=None, engine="python", encoding="utf-8")
sub.columns = [c.strip() for c in sub.columns]
sub["word_norm"] = sub["spelling"].map(_norm_word) if "spelling" in sub.columns else sub.iloc[:, 0].map(_norm_word)

zipf_col = None
for cand in ["zipf.freq", "zipf_freq", "zipf", "Zipf", "avg.zipf.freq.sn", "zipf.freq.sn.sum"]:
    if cand in sub.columns:
        zipf_col = cand
        break

anpw = pd.read_csv(ANPW_FILE, sep=None, engine="python", encoding="utf-8")
anpw.columns = [c.strip() for c in anpw.columns]
anpw_word_col = "polish word" if "polish word" in anpw.columns else anpw.columns[1]
anpw["word_norm"] = anpw[anpw_word_col].map(_norm_word)

def to_float_series(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s.astype(str).str.replace(",", ".", regex=False), errors="coerce")

for col in ["concretness_M", "Valence_M", "arousal_M", "ageOfAquisition_M", "Number of Letters"]:
    if col in anpw.columns:
        anpw[col] = to_float_series(anpw[col])

lists["nchar"] = lists["word"].astype(str).str.len()
lists["syllables"] = lists["word"].map(count_syllables_pl)

feat = lists.merge(
    anpw[["word_norm"] + [c for c in ["concretness_M", "Valence_M", "arousal_M", "ageOfAquisition_M", "Number of Letters"] if c in anpw.columns]],
    on="word_norm",
    how="left",
)

if zipf_col is not None:
    feat = feat.merge(sub[["word_norm", zipf_col]], on="word_norm", how="left")
    feat = feat.rename(columns={zipf_col: "zipf"})
else:
    feat["zipf"] = np.nan

feat.head()


,word,condition,run,stimFile,list_name,word_norm,nchar,syllables,concretness_M,Valence_M,arousal_M,ageOfAquisition_M,Number of Letters,zipf
0,zegar,CON,1,resources/audio/con_run1_001.wav,con_run1,zegar,5,2,1.96,5.12,3.02,6.14,5.0,4.050517
1,samochód,CON,1,resources/audio/con_run1_002.wav,con_run1,samochód,8,3,1.88,5.92,3.86,6.96,8.0,5.210732
2,grzebień,CON,1,resources/audio/con_run1_003.wav,con_run1,grzebień,8,2,1.82,5.26,3.34,7.12,8.0,3.179123
3,krowa,CON,1,resources/audio/con_run1_004.wav,con_run1,krowa,5,2,1.76,5.08,2.94,6.26,5.0,3.883150
4,piasek,CON,1,resources/audio/con_run1_005.wav,con_run1,piasek,6,2,1.96,5.92,3.04,6.58,6.0,3.854373


In [ ]:
# sanity checks

feat.groupby(["run", "condition"])["word_norm"].nunique().rename("n_unique_words").reset_index()


,run,condition,n_unique_words
0,1,ABS,24
1,1,CON,24
2,2,ABS,24
3,2,CON,24


In [6]:
CONTROL_VARS = ["nchar", "syllables", "zipf", "Valence_M", "arousal_M", "ageOfAquisition_M"]
MANIP_VAR = "concretness_M"

def run_tests(df: pd.DataFrame, var: str) -> dict:
    x = df.loc[df["condition"] == "CON", var].astype(float).to_numpy()
    y = df.loc[df["condition"] == "ABS", var].astype(float).to_numpy()
    x = x[~np.isnan(x)]
    y = y[~np.isnan(y)]

    out = {
        "var": var,
        "n_CON": len(x),
        "n_ABS": len(y),
        "mean_CON": np.mean(x) if len(x) else np.nan,
        "mean_ABS": np.mean(y) if len(y) else np.nan,
        "sd_CON": np.std(x, ddof=1) if len(x) > 1 else np.nan,
        "sd_ABS": np.std(y, ddof=1) if len(y) > 1 else np.nan,
    }

    if len(x) and len(y):
        ks = stats.ks_2samp(x, y, alternative="two-sided", mode="auto")
        out["ks_stat"] = float(ks.statistic)
        out["ks_p"] = float(ks.pvalue)
    else:
        out["ks_stat"] = np.nan
        out["ks_p"] = np.nan

    if len(x) > 1 and len(y) > 1:
        lev = stats.levene(x, y, center="median")
        out["levene_stat"] = float(lev.statistic)
        out["levene_p"] = float(lev.pvalue)
    else:
        out["levene_stat"] = np.nan
        out["levene_p"] = np.nan

    if len(x) > 1 and len(y) > 1:
        tt = stats.ttest_ind(x, y, equal_var=False, nan_policy="omit")
        out["t_stat_welch"] = float(tt.statistic)
        out["t_p_welch"] = float(tt.pvalue)
    else:
        out["t_stat_welch"] = np.nan
        out["t_p_welch"] = np.nan

    if len(x) and len(y):
        mw = stats.mannwhitneyu(x, y, alternative="two-sided")
        out["mw_u"] = float(mw.statistic)
        out["mw_p"] = float(mw.pvalue)
    else:
        out["mw_u"] = np.nan
        out["mw_p"] = np.nan

    out["cohen_d_CON_minus_ABS"] = float(cohen_d(x, y))
    lo, hi = bootstrap_ci_d(x, y, n_boot=3000, seed=202)
    out["d_ci95_low"] = lo
    out["d_ci95_high"] = hi
    return out

def make_report(df: pd.DataFrame, label: str) -> pd.DataFrame:
    rows = []
    for v in CONTROL_VARS + [MANIP_VAR]:
        if v in df.columns:
            rows.append(run_tests(df, v))
    rep = pd.DataFrame(rows)
    rep.insert(0, "scope", label)
    return rep


In [7]:
rep_run1 = make_report(feat[feat["run"] == 1].copy(), "run1")
rep_run2 = make_report(feat[feat["run"] == 2].copy(), "run2")
rep_all = make_report(feat.copy(), "all_runs")

rep_run1, rep_run2, rep_all


(  scope                var  n_CON  n_ABS  mean_CON  mean_ABS    sd_CON  \
 0  run1              nchar     24     24  6.916667  6.958333  2.062431   
 1  run1          syllables     24     24  2.416667  2.416667  0.829702   
 2  run1               zipf     23     23  3.966457  4.012158  0.596968   
 3  run1          Valence_M     24     24  5.533333  5.211667  0.817497   
 4  run1          arousal_M     24     24  3.480000  4.261667  0.569241   
 5  run1  ageOfAquisition_M     24     24  7.083333  9.052500  1.136075   
 6  run1      concretness_M     24     24  1.870000  5.471667  0.116171   
 
      sd_ABS   ks_stat          ks_p  levene_stat      levene_p  t_stat_welch  \
 0  2.136213  0.083333  9.999995e-01     0.143451  7.066165e-01     -0.068744   
 1  0.829702  0.000000  1.000000e+00     0.000000  1.000000e+00      0.000000   
 2  0.620074  0.130435  9.923772e-01     0.000131  9.909135e-01     -0.254636   
 3  1.454707  0.291667  2.628338e-01     6.289957  1.573320e-02      0.944

In [8]:
rep_run1.to_csv(REPORTS_DIR / "stimuli_report_run1.csv", index=False, encoding="utf-8-sig")
rep_run2.to_csv(REPORTS_DIR / "stimuli_report_run2.csv", index=False, encoding="utf-8-sig")
rep_all.to_csv(REPORTS_DIR / "stimuli_report_all.csv", index=False, encoding="utf-8-sig")

print("Saved:")
print(" -", REPORTS_DIR / "stimuli_report_run1.csv")
print(" -", REPORTS_DIR / "stimuli_report_run2.csv")
print(" -", REPORTS_DIR / "stimuli_report_all.csv")


Saved:
 - C:\Users\kinga\Documents\Blindbrain\4. Courses\fMRI - design of the experiment and data analysis\cognes-auditory-1back-pilot\stimuli_selection\reports\stimuli_report_run1.csv
 - C:\Users\kinga\Documents\Blindbrain\4. Courses\fMRI - design of the experiment and data analysis\cognes-auditory-1back-pilot\stimuli_selection\reports\stimuli_report_run2.csv
 - C:\Users\kinga\Documents\Blindbrain\4. Courses\fMRI - design of the experiment and data analysis\cognes-auditory-1back-pilot\stimuli_selection\reports\stimuli_report_all.csv


In [9]:
def summarize(rep: pd.DataFrame, scope: str) -> None:
    r = rep[rep["scope"] == scope].set_index("var")

    def fmt_p(p):
        return f"{p:.3g}" if pd.notna(p) else "NA"

    print(f"== {scope} ==")
    for v in CONTROL_VARS:
        if v not in r.index:
            continue
        print(f"{v}: KS p={fmt_p(r.loc[v,'ks_p'])}, Levene p={fmt_p(r.loc[v,'levene_p'])}, Welch p={fmt_p(r.loc[v,'t_p_welch'])}")
    if MANIP_VAR in r.index:
        d = r.loc[MANIP_VAR, "cohen_d_CON_minus_ABS"]
        lo = r.loc[MANIP_VAR, "d_ci95_low"]
        hi = r.loc[MANIP_VAR, "d_ci95_high"]
        print(f"{MANIP_VAR}: d={d:.2f} [{lo:.2f}, {hi:.2f}], Welch p={fmt_p(r.loc[MANIP_VAR,'t_p_welch'])}")

summarize(rep_all, "all_runs")


== all_runs ==
nchar: KS p=1, Levene p=0.932, Welch p=0.958
syllables: KS p=1, Levene p=1, Welch p=1
zipf: KS p=1, Levene p=0.878, Welch p=0.757
Valence_M: KS p=0.371, Levene p=0.000403, Welch p=0.998
arousal_M: KS p=8.29e-07, Levene p=0.14, Welch p=1.31e-07
ageOfAquisition_M: KS p=6.72e-08, Levene p=0.03, Welch p=2.04e-12
concretness_M: d=-5.67 [-6.71, -5.05], Welch p=1.27e-31


In [ ]:
# distributions saved as PNG

PLOT_VARS = ["nchar", "syllables", "zipf", "concretness_M"]

def plot_var(df: pd.DataFrame, var: str, label: str) -> None:
    d = df[["condition", var]].dropna()
    x = d.loc[d["condition"] == "CON", var].astype(float).to_numpy()
    y = d.loc[d["condition"] == "ABS", var].astype(float).to_numpy()

    plt.figure(figsize=(7, 4))
    plt.hist(x, bins=10, alpha=0.6, label="CON")
    plt.hist(y, bins=10, alpha=0.6, label="ABS")
    plt.title(f"{label}: {var}")
    plt.xlabel(var)
    plt.ylabel("count")
    plt.legend()
    out = FIG_DIR / f"{label}_{var}.png"
    plt.tight_layout()
    plt.savefig(out, dpi=150)
    plt.close()

for label, df_ in [("run1", feat[feat["run"] == 1]), ("run2", feat[feat["run"] == 2]), ("all", feat)]:
    for v in PLOT_VARS:
        if v in df_.columns:
            plot_var(df_, v, label)

print("Saved figures to:", FIG_DIR)


Saved figures to: C:\Users\kinga\Documents\Blindbrain\4. Courses\fMRI - design of the experiment and data analysis\cognes-auditory-1back-pilot\stimuli_selection\reports\figures
